In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [2]:
from typing import TypedDict, Optional

class AuthState(TypedDict):
    username: Optional[str] 
    password: Optional[str]
    is_authenticated: Optional[bool]
    output: Optional[str]

In [3]:
def input_node(state):
    print(state)
    if state.get('username', "") =="":
        state['username'] = input("What is your username?")

    password = input("Enter your password: ")

    return {"password":password}

In [4]:
auth_state_1: AuthState = {
    "username": "alice123",
    "password": "123",
    "is_authenticated": True,
    "output": "Login successful."
}
print(f"auth_state_1: {auth_state_1}")

auth_state_1: {'username': 'alice123', 'password': '123', 'is_authenticated': True, 'output': 'Login successful.'}


In [5]:
input_node(auth_state_1)

{'username': 'alice123', 'password': '123', 'is_authenticated': True, 'output': 'Login successful.'}


{'password': 'secure_password'}

In [6]:
def validate_credentials_node(state):
    # Extract username and password from the state
    username = state.get("username", "")
    password = state.get("password", "")

    print("Username :", username, "Password :", password)
    # Simulated credential validation
    if username == "test_user" and password == "secure_password":
        is_authenticated = True
    else:
        is_authenticated = False

    # Return the updated state with authentication result
    return {"is_authenticated": is_authenticated}

In [7]:
validate_credentials_node(auth_state_1)

Username : alice123 Password : 123


{'is_authenticated': False}

In [8]:
# Define the success node
def success_node(state):
    return {"output": "Authentication successful! Welcome."}

In [9]:
def router(state):
    if state['is_authenticated']:
        return "success_node"
    else:
        return "failure_node"

In [10]:
from langgraph.graph import StateGraph
from langgraph.graph import END

# Create an instance of StateGraph with the GraphState structure
workflow = StateGraph(AuthState)
workflow

In [ ]:
workflow.add_node("InputNode", input_node)
workflow.add_node("ValidateCredential", validate_credentials_node)
workflow.add_node("SuccessNode", success_node)
workflow.add_node("FailureNode", lambda state: {"output": "Authentication failed. Please try again."})
workflow.add_edge("InputNode", "ValidateCredential")
workflow.add_edge("SuccessNode", END)
workflow.add_edge("FailureNode", "InputNode")